In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision import models

Task1 
1. Load CIFAR1-10 dataset

In [8]:
# 1. Load CIFAR-10 Dataset
transform = transforms.ToTensor()
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

In [17]:
class SimpleCNN(nn.Module):
    def __init__(self, activation_fn=nn.LeakyReLU()):
        super(SimpleCNN, self).__init__()
        # The first layer has 3 input channels and 32 output channels. The point here is
        # that the CIFR images are 32 * 32, and RGB channels after the to tensor. So, we take
        self.conv_layer = nn.Conv2d(3, 32, kernel_size=5) 
        # Fully connected layer - the hidden layers are 28 * 28 hence and pooling reduces it by 2
        # Hence, 14 * 14 and mult that with 32 feature/conv layers.
        self.nn_layer = nn.Linear(32 * 14 * 14, 10)
        self.activation = activation_fn  # Activation function
        self.pool = nn.MaxPool2d(2, 2)  # Max pooling layer
    
    def forward(self, x):
        x = self.pool(self.activation(self.conv_layer(x))) 
        x = torch.flatten(x, 1)  # Flatten the feature maps, as the next layer is 1d layer
        x = self.nn_layer(x)  # Output layer
        return x

# ANN-1 excercise
def train_model(model, optimizer, error_func, device, epochs=10, log_dir=""):
    writer = SummaryWriter(log_dir)
    model.to(device)
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        for images, labels in trainloader:
            
            images, labels = images.to(device), labels.to(device) 
            optimizer.zero_grad()  # Reset gradients
            outputs = model(images)  # Forward pass
            loss = error_func(outputs, labels)  # Compute loss
            loss.backward()  # Backpropagation
            optimizer.step()  # Update weights
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)  # Get predictions
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        train_acc = 100 * correct / total  # Compute accuracy
        writer.add_scalar('Loss/train', running_loss / len(trainloader), epoch)  # Log loss
        writer.add_scalar('Accuracy/train', train_acc, epoch)  # Log accuracy
        print(f'Epoch {epoch+1}, Loss: {running_loss / len(trainloader)}, Accuracy: {train_acc}%')
    writer.close()

# Train model with LeakyReLU and SGD optimizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"The device I have is {device}")

The device I have is cuda


In [10]:


model1 = SimpleCNN(nn.LeakyReLU())
optimizer1 = optim.SGD(model1.parameters(), lr=0.0001)
err_func = nn.CrossEntropyLoss()
train_model(model1, optimizer1, err_func,device, 10, log_dir='logs/leakyrelu_sgd')

# Train model with LeakyReLU and Adam optimizer
model2 = SimpleCNN(nn.LeakyReLU())
optimizer2 = optim.Adam(model2.parameters())
train_model(model2, optimizer2, err_func,device, 10, log_dir='logs/leakyrelu_adam')

# Train model with Tanh and Adam optimizer
model3 = SimpleCNN(nn.Tanh())
optimizer3 = optim.Adam(model3.parameters())
train_model(model3, optimizer3, err_func,device, 10, log_dir='logs/tanh_adam')



Epoch 1, Loss: 2.3018196123030483, Accuracy: 12.056%
Epoch 2, Loss: 2.283367352717368, Accuracy: 14.964%
Epoch 3, Loss: 2.2679799975031782, Accuracy: 18.66%
Epoch 4, Loss: 2.2523631528210455, Accuracy: 21.394%
Epoch 5, Loss: 2.2362577613357386, Accuracy: 22.874%
Epoch 6, Loss: 2.2195140336785477, Accuracy: 24.066%
Epoch 7, Loss: 2.202300337574366, Accuracy: 24.944%
Epoch 8, Loss: 2.1845530448362345, Accuracy: 25.92%
Epoch 9, Loss: 2.167002440718434, Accuracy: 26.12%
Epoch 10, Loss: 2.14963867017985, Accuracy: 26.972%
Epoch 1, Loss: 1.507934270612419, Accuracy: 47.094%
Epoch 2, Loss: 1.230274350518156, Accuracy: 57.606%
Epoch 3, Loss: 1.124247382097232, Accuracy: 61.48%
Epoch 4, Loss: 1.0578128779330827, Accuracy: 63.616%
Epoch 5, Loss: 1.0119071758311728, Accuracy: 65.146%
Epoch 6, Loss: 0.9736140230884942, Accuracy: 66.57%
Epoch 7, Loss: 0.9415218439096075, Accuracy: 67.808%
Epoch 8, Loss: 0.9150028026393612, Accuracy: 68.588%
Epoch 9, Loss: 0.8918096398758462, Accuracy: 69.412%
Epoch

In [20]:
# 2. Fine-tuning the AlexNet model for CIFAR-10
transform_alex = transforms.Compose([
    transforms.Resize(224),  # Resize to 224x224 to match AlexNet input size
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5, 0.5, 0.5],
        std=[0.5, 0.5, 0.5]
    )])


trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_alex)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_alex)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

alexnet = models.alexnet(pretrained=False)  # We will initialize AlexNet without pre-trained weights
alexnet.classifier[6] = nn.Linear(in_features=4096, out_features=10) 
optimizer_alexnet = optim.Adam(alexnet.parameters(), lr=0.001)
err_func = nn.CrossEntropyLoss()
# device = torch.device("cpu")

train_model(alexnet, optimizer_alexnet, err_func,device, 10, log_dir='logs/alexnet_finetuning')

alexnet_pretrained = models.alexnet(pretrained=True)


alexnet_pretrained.classifier[6] = nn.Linear(in_features=4096, out_features=10)
optimizer_pretrained = optim.Adam(alexnet_pretrained.classifier[6].parameters())

train_model(alexnet_pretrained, optimizer_pretrained, err_func,device, 5, log_dir='logs/alexnet_feature_extraction')

Epoch 1, Loss: 1.749811439715383, Accuracy: 34.37%
Epoch 2, Loss: 1.3875869479020844, Accuracy: 49.598%
Epoch 3, Loss: 1.2480666821112718, Accuracy: 55.462%
Epoch 4, Loss: 1.1492799949615509, Accuracy: 59.346%
Epoch 5, Loss: 1.0784562936676738, Accuracy: 61.984%
Epoch 6, Loss: 1.0219819440561182, Accuracy: 64.004%
Epoch 7, Loss: 0.9642485929724506, Accuracy: 65.928%
Epoch 8, Loss: 0.9198754125696313, Accuracy: 67.728%
Epoch 9, Loss: 0.8928725300999858, Accuracy: 68.604%
Epoch 10, Loss: 0.8593373371435858, Accuracy: 69.632%


d:\LTU\AdvancedNN\ann_lab_0\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1, Loss: 0.8791558714321507, Accuracy: 69.098%
Epoch 2, Loss: 0.779891442612309, Accuracy: 72.726%
Epoch 3, Loss: 0.7589350004330315, Accuracy: 73.352%
Epoch 4, Loss: 0.7487478452494077, Accuracy: 73.59%
Epoch 5, Loss: 0.7383266522375214, Accuracy: 74.126%


In [21]:
#Using the same CNN for MINST and SVHN datsets.


transform_mnist = transforms.Compose([
    transforms.Resize((32, 32)),  # Resize to match your model's expected input size
    transforms.Grayscale(num_output_channels=3),  # Convert 1 channel -> 3 channels
    transforms.ToTensor()
])

mnist_trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform_mnist)
mnist_testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform_mnist)

trainloader = DataLoader(mnist_trainset, batch_size=64, shuffle=True)
testloader = DataLoader(mnist_testset, batch_size=64, shuffle=False)


model_minst = SimpleCNN(nn.Tanh())
optimizer_minst = optim.Adam(model_minst.parameters())

# optimizer1 = optim.SGD(model1.parameters(), lr=0.0001)
err_func = nn.CrossEntropyLoss()
train_model(model_minst, optimizer_minst, err_func,device, 5, log_dir='logs/minst_tanh_adam')

Epoch 1, Loss: 0.22566888012540048, Accuracy: 93.35%
Epoch 2, Loss: 0.08500903119136498, Accuracy: 97.435%
Epoch 3, Loss: 0.06205552467815221, Accuracy: 98.08166666666666%
Epoch 4, Loss: 0.052215030739656776, Accuracy: 98.34%
Epoch 5, Loss: 0.04461047726684311, Accuracy: 98.60666666666667%


In [24]:
transform_svhn = transforms.Compose([transforms.ToTensor()])

svhn_trainset = torchvision.datasets.SVHN(root='./data', split='train', download=True, transform=transform_svhn)
svhn_testset = torchvision.datasets.SVHN(root='./data', split='test', download=True, transform=transform_svhn)

svhn_trainloader = DataLoader(svhn_trainset, batch_size=64, shuffle=True)
svhn_testloader = DataLoader(svhn_testset, batch_size=64, shuffle=False)

model_svhn = SimpleCNN(nn.Tanh())
optimizer_svhn = optim.Adam(model_svhn.parameters())

# optimizer1 = optim.SGD(model1.parameters(), lr=0.0001)
err_func = nn.CrossEntropyLoss()
train_model(model_svhn, optimizer_svhn, err_func,device, 5, log_dir='logs/svhn_tanh_adam')

100.0%


Epoch 1, Loss: 0.2139931849803704, Accuracy: 93.75833333333334%
Epoch 2, Loss: 0.08197729043942541, Accuracy: 97.51833333333333%
Epoch 3, Loss: 0.0652031527945339, Accuracy: 97.96%
Epoch 4, Loss: 0.05285450167157677, Accuracy: 98.40333333333334%
Epoch 5, Loss: 0.04694962119831314, Accuracy: 98.5%
